# Querying

> __Prometheus has a lot of querying capabilities (including it's own PromQL language!) which we will learn about today__

- __Please start an instance of `node` & some docker containers Prometheus server (inside docker container) just like we did in previous lesson!__
- __Go to `localhost:9090` to have query/expression prompt__ 


## Data types

Only one of the four listed below are allowed:
- `float64` (any scalar is a `float`!)
- `string` (unused by `prometheus` currently)
- __instant vectors__
- __range vectors__

## Instant Vector selectors

> __Instant vectors are created via SELECTION based on some matching patterns (labels, metric names etc.)__

__Easiest form is a simple `metric` name statement returning vector of values.__

Try typing this in the expressions field on your Prometheus dashboard. Some of these metrics might appear differently depending on your operating system.

In [ ]:
node_cpu_seconds_total

In [ ]:
# Windows 
windows_cpu_time_total

![](images/prometheus_vector_dtype_basic.png)

We can further filter the metric by specific labels, for example:

In [ ]:
node_cpu_seconds_total{cpu=~"0|1", instance!="localhost:8081", mode="user"}

In [ ]:
# on windows
windows_cpu_time_total{core=~"0,0|0,1", job="wmiexporter"}

![](images/prometheus_node_query_matching.png)

### Matching

> Prometheus supports different comparison operators __and regex matching__ (via [Google's RE2 syntax](https://github.com/google/re2/wiki/Syntax))

__Comparison:__
- `=` - labels which are equal to
- `!=` - labels which __are not__ equal to
- `=~` - labels which __regex match__ string
- `!~` - labels which __regex UNmatch__ string

Other than that, regex matching works pretty similar to how it usually does (check out specification linked above if in doubt).

### __name__

Similar to Python's `__name__` Prometheus also provides this `label` as an internal one. __It allows us to match on expressions__, for example:

A quick way to match strings is using the `".*"` expression, the `.` signifies matching any character and the `*` signifies matching any number of characters afterwards. 

In [ ]:
{__name__=~"node.*_seconds"}

In [ ]:
{__name__=~"windows.*_time_.*"}

![](images/prometheus_name_match.png)

One thing you have to keep in mind:

> __Regex matching HAS TO MATCH something, YOUR EXPRESSION HAS TO BE VALID__


For example (label matching every job __AND EMPTY JOB ALSO__ where job starts with wmi):

In [ ]:
{job=~"wmi.*"}

While the `+` here will match any job that doesn't have an empty string. 

In [ ]:
{job=~".+"}

## Range vector selectors

> __Range vectors are created by slicing timeseries based on time duration__

Previously we used `{}` for instant selectors, time time we will use `[]`: 

In [ ]:
node_cpu_seconds_total{cpu=~"0|1", mode="idle"}[20s]

In [ ]:
# Windows
windows_cpu_time_total{core=~"0,.*|0,.*", job="wmiexporter"}[20s]

> Range vector __CANNOT BE GRAPHED__ (we have to transform them to instant vectors via some function) __BECAUSE THEY HAVE MULTIPLE VALUES FOR A SINGLE TIMESTAMP__

![](images/prometheus_range_selector.png)

We can see that __now we have a multidimensionally aggregated data__:
- We get `4` values for each timeseries
- `4` values for `20s` range __because data is scrapped every `5s`__

Given that, we can perform some kind of operation on grouped data.

To plot the above, we can summarize the data, for example using the following query:

In [ ]:
rate(node_cpu_seconds_total{cpu=~"0|1", mode="idle"}[5m])

In [ ]:
rate(windows_cpu_time_total{core=~"0,.*|0,.*", mode="idle"}[5m])

![](images/prometheus_range_rate_cpus.png)

There are a few available time units for our usage:

- `ms` - milliseconds
- `s` - seconds
- `m` - minutes
- `h` - hours
- `d` - days - assuming a day has always 24h
- `w` - weeks - assuming a week has always 7d
- `y` - years - assuming a year has always 365d

> One can mix the above __but they have to be ordered from largest to smallest unit__

For example:
- `1h30m`
- `2d3h15m10s7ms`

### Offsets

> __Offsets allow us to jump to specified point in time__

For example, query below could return `5` minute rate of `http_requests` we had `1` week ago:

In [ ]:
rate(http_requests_total[5m] offset 1w)

## Operators

> __Prometheus's language (PromQL) provides standard set of operators (logical, arithmetical etc.)__

| Arithmetic        | Comparison          | Logical  |
| ------------- |:-------------:| -----:|
| + | == | and |
| - | != | or |
| / | > | unless (complement) |
| \% | <  | |
| ^ | >=  | |
|  | <=   | |


Those follow standard broadcasting rules between scalars and vectors we already know from `numpy` or `pytorch`, __except matching between two `instant vectors`__: 

## Vector matching

> Vector matching defines one `instant vector` can be matched to another one

### One-to-one

> One to one finds a unique pair of entries from each side of the operation.

Here are the governing rules:
- exact same set of labels
- value types have to match

In [ ]:
<vector expr> <op> ignoring(<label list>) <vector expr>
<vector expr> <op> on(<label list>) <vector expr>

- __ignoring__ allows us to ignore label(s)
- __on__ allows us to specify labels

Let's assume we have the following two groups of timeseries:
- `method_code:http_errors:rate5m` - `5m` rate of `http_errors` and their specific `code`
- `method:http_requests:rate5m` - `5m` rate of `http_errors` for specific method

Now, for specific `instant selectors` we might have (example values at a given timestamp are commented out on the right):

In [ ]:
method_code:http_errors:rate5m{method="get", code="500"}  # 24
method_code:http_errors:rate5m{method="get", code="404"}  # 30
method_code:http_errors:rate5m{method="put", code="501"}  # 3
method_code:http_errors:rate5m{method="post", code="500"} # 6
method_code:http_errors:rate5m{method="post", code="404"} # 21

method:http_requests:rate5m{method="get"}  # 600
method:http_requests:rate5m{method="del"}  # 34
method:http_requests:rate5m{method="post"} # 120

Now, let's see what is the ratio of failed requests with `code=500` (internal server error rate):

In [ ]:
method_code:http_errors:rate5m{code="500"} / ignoring(code) method:http_requests:rate5m

This would match:
- `method_code:http_errors:rate5m{method="get", code="500"}` with `method:http_requests:rate5m{method="get"} ` (notice labels match, code doesn't __but we are ignoring it__), returning value `24 / 600 = 0.04`
- `method_code:http_errors:rate5m{method="post", code="500"}` with `method:http_requests:rate5m{method="post"} ` - same thing as above, returning value `6 / 120 = 0.05`

Hence we would obtain two `instant vectors` 

### Many-to-one & one-to-many

> __Each vector element on the "one"-side can match with multiple elements on the "many"-side.__

This use case is advanced and given as an __mandatory__ exercise.

__Note:__ You should use this approach only when absolutely necessary!

## Aggregation operations

> Prometheus provides basic operations for data aggregation

Full list:

- `sum` - calculate sum over dimensions
- `min` - select minimum over dimensions
- `max` - select maximum over dimensions
- `avg` - calculate the average over dimensions
- `group` - all values in the resulting vector are 1
- `stddev` - calculate population standard deviation over dimensions
- `stdvar` - calculate population standard variance over dimensions
- `count` - count number of elements in the vector
- `count_values` - count number of elements with the same value
- `bottomk` - smallest k elements by sample value
- `topk` - largest k elements by sample value
- `quantile` - calculate φ-quantile (0 ≤ φ ≤ 1) over dimensions

Simplest form is:

```
<op>(<vector_expression>)
```


In [ ]:
min(node_cpu_scaling_frequency_hertz)

In [ ]:
min(windows_cpu_core_frequency_mhz)

![](images/prometheus_op_simple.png)

In this case __minimum value for each timestep is taken across ALL OF THE LABELS__.

To specify labels (dimensions) should the operation be run across, we have two modifiers:
- `by` - specifies __by which label(s) the `min` is taken__ (all unspecified will be "flattened")
- `without` - specifies __without which label(s) the `min` is taken__ (all specified will be "flattened")

Their syntax is as folows:

```
<op> by (<label>) (<vector_expression>)
```

or:

```
<op> without (<label>) (<vector_expression>)
```

`min` taken for each `cpu` separately would be:

In [ ]:
min by (cpu) (node_cpu_scaling_frequency_hertz)

min by (core) (windows_cpu_core_frequency_mhz)

![](images/prometheus_op_by.png)

## Functions

> __Prometheus provides a set of functions one can use (full list [here](https://prometheus.io/docs/prometheus/latest/querying/functions/))__

Currently this set is a little limited __and you cannot add your own new functions__ (at least without forking `prometheus` project).

- Standard rules apply (e.g. some function have default arguments)
- Some operate on range vectors, while others on instant vectors

Here are some groups of functions:
- `date` related (`minute`, `month`, `year`, `timestamp`, `day_of_month` etc.)
- `math` related (`ln`, `exp`, `deriv`, `round`, `sgn` etc.)
- `timeseries` related (`delta` (difference between consecutive values), `idelta` etc.)

Below are a few examples with descriptions as comments:

In [2]:
# Monitored hardware temperature difference of `10m` intervals
# First and last value from those intervals will be taken

delta(node_hwmon_temp_celsius[10m])

bash: syntax error near unexpected token `node_hwmon_temp_celsius[10m]'


: 2

In [ ]:
delta(windows_net_packets_received_total[10m])

![](images/prometheus_temp_delta.png)

In [ ]:
# Calculates increase between last value in range vector and the first
# Adjusted for monotonicity and smoothed out

increase(node_hwmon_temp_celsius[10m])

In [ ]:
increase(windows_net_packets_received_total[10m])

![](images/prometheus_increase_function.png)

In [ ]:
# e^temp exponential value
# This one operates on instant vectors

exp(node_hwmon_temp_celsius)

In [ ]:
exp(windows_cpu_time_total)

![](images/prometheus_exp_function.png)

### {op}_over_time

Last set of provided Prometheus functions is given by the following scheme :

```
{op}_over_time(v range-vector)
```

where you put different `op`s like `avg`, `min` and the operation is carried across specified `range`s.

Check out all possibilities [here](https://prometheus.io/docs/prometheus/latest/querying/functions/#aggregation_over_time) and an example below:

In [ ]:
stddev_over_time(windows_system_context_switches_total[10s])

![](images/prometheus_stddev_over_time.png)

## Recording rules

> __Recording rules allow us to precompute frequently used/expensive and save results as a new timeseries__

__You should always try to create new rules instead of running ad-hoc commands__, as:
- Faster, as they work on less data but on a regular basis
- More readable for others
- Easy to reuse
- Easy to put in VCS systems like `git`

Rules are usually writen in separate `.yml` file (`<name>.rules.yml` seems like a reasonable choice) __and included in `prometheus.yml` config__ we have seen in a previous lesson.

Let's take a look at the `example.rules.yml`:

In [ ]:
groups: # High level grouping
  - name: example # Name of the group of rules
    interval: 30s # How often they should be evaluated (deafult: 1m)
    rules: # Set of rules in this this group
    - record: job:http_inprogress_requests:sum # Name of the rule
      expr: sum by (job) (http_inprogress_requests) # Evaluated expression

### Groups

> Rules within a group are run sequentially (as defined) in a regular interval

Those should be grouped by:
- semantic meaning
- evaluation interval

### Naming

There a few guidelines you should stick to:
- Recording rules should be of the general form `level:metric:operations`:
    - `level` - labels of the rule output / aggregation level
    - `metric` - name of the metric, e.g. `http_requests`
    - `operations` - key operations creating the result
    
Some examples:

In [ ]:
- record: instance_path:requests:rate5m
  expr: rate(requests_total{job="myjob"}[5m])

- record: path:requests:rate5m
  expr: sum without (instance)(instance_path:requests:rate5m{job="myjob"})
  
- record: instance_path:request_failures:rate5m
  expr: rate(request_failures_total{job="myjob"}[5m])

- record:  wmiexporter:windows_cpu_dpcs_total:sum 
  expr: sum by (job) (windows_cpu_dpcs_total)  

### Including rules in prometheus.yml

After we have our rule written down, __we have to include it in `prometheus.yml` server config__.

There are two places where one can change related `recording rules` settings:

In [ ]:
global:
  # How frequently to evaluate rules.
  # Define on a per-group basis if needed
  [ evaluation_interval: <duration> | default = 1m ]
rule_files:
  [ - <filepath_glob> ... ] # Path to rule files 

For example:

In [ ]:
# my global config
global:
  evaluation_interval: 5m # Evaluate rules every 15 seconds. The default is every 1 minute.

# Load rules once and periodically evaluate them according to the global 'evaluation_interval'.
rule_files:
   - "windows.rules.yml"
   - "docker.rules.yml"

...

After those steps, your rules will run automatically and be available under the `record` name you specified!

## Challenges


### Assessment

- Check out [best Prometheus practices](https://prometheus.io/docs/practices/) and do your best 
- Check out [more functions examples](https://prometheus.io/docs/prometheus/latest/querying/examples/) to get a better feel of what one can do with it
- What is `@` modifier and how to use it? See [here](https://prometheus.io/docs/prometheus/latest/querying/basics/#modifier)
- How to use `many-to-one` and `one-to-many`? Read [here](https://prometheus.io/docs/prometheus/latest/querying/operators/#many-to-one-and-one-to-many-vector-matches)

### Non-assessment

- What is Prometheus's `promtool`? How can it help you?
- How to create [Alerting rules](https://prometheus.io/docs/prometheus/latest/configuration/alerting_rules/)? 